<a href="https://colab.research.google.com/github/WesleySantosMaxtelll/doutorado/blob/main/mcnemar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
import pandas as pd
import pickle
import os
from sklearn.metrics import classification_report, matthews_corrcoef, roc_auc_score
from collections import Counter
import numpy as np

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

def main_mcnemar(c1, c2):
    df = pd.concat([c1,c2],axis=1) # lado
    df.columns = ['c1','c2']

    df['right_right'] = df.apply(lambda x: int(x.c1==1 and x.c2==1),axis=1)
    df['right_wrong'] = df.apply(lambda x: int(x.c1==1 and x.c2==0),axis=1)
    df['wrong_right'] = df.apply(lambda x: int(x.c1==0 and x.c2==1),axis=1)
    df['wrong_wrong'] = df.apply(lambda x: int(x.c1==0 and x.c2==0),axis=1)

    cell_11 = df.right_right.sum()
    cell_12 = df.right_wrong.sum()
    cell_21 = df.wrong_right.sum()
    cell_22 = df.wrong_wrong.sum()
    
    table = [
             [cell_11, cell_12],
             [cell_21, cell_22] 
            ]
                
    if cell_11<25 or cell_12<25 or cell_21<25 or cell_22<25:
        result = mcnemar(table, exact=True)
    else:
        result = mcnemar(table, exact=False, correction=True)   

    # print('\n'+corpus+"_"+classifier1+"_"+classifier2+ ' : ',end='')
    print('stat=%.3f, p=%.7f' % (result.statistic, result.pvalue),end='')

    significant = False
    for alpha in [0.001, 0.01, 0.05]:
        if result.pvalue < alpha:
            print(' p<' + str(alpha))
            significant = True
            break
    if not significant:
        print(' not significant')


In [ ]:
df1 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/prediction/D_pre.pkl', 'rb'))
df2 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/prediction/D_pos.pkl', 'rb'))

In [ ]:
df1 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/ens-concat_all_anxiety.pkl', 'rb'))
df2 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/moe_all_anxiety.pkl', 'rb'))

In [ ]:
df1 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/ens-concat_all_depression.pkl', 'rb'))
df2 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/moe_all_depression.pkl', 'rb'))

In [ ]:
df1 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/lstm_treatment_anxiety.pkl', 'rb'))
df2 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/lstm_diagnosis_anxiety.pkl', 'rb'))

In [ ]:
df1 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/all_anxiety.pkl', 'rb'))
df2 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/rl/anxiety-rl-tfidf.pkl', 'rb'))

In [ ]:
df1 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/all_anxiety.pkl', 'rb'))
df2 = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/rl/anxiety-rl-tfidf.pkl', 'rb'))

{'pred': array([0, 1, 0, ..., 0, 1, 1]),
 'true': array([0, 0, 0, ..., 1, 1, 1])}

In [ ]:
df1 = pd.read_csv(f'DEM_test_prediction_semeval_A.csv')
df2 = pd.read_csv(f'test_prediction_semeval_2016.csv')

In [ ]:
def clean_df(df):
    df = df[df['real']!=2]
    def conv(x):
        if x['pred'] == 2:
            if x['real'] == 1:
                return 0
            else:
                return 1
        return x['pred']
    df['pred'] = df.apply(lambda x: conv(x), axis=1)
    return df

In [ ]:
df1 = clean_df(df1)
df2 = clean_df(df2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
print(classification_report(df1['real'], df1['pred']))
print(classification_report(df2['real'],df2['pred']))

              precision    recall  f1-score   support

           0       0.92      0.85      0.89      1547
           1       0.32      0.48      0.39       221

    accuracy                           0.81      1768
   macro avg       0.62      0.67      0.64      1768
weighted avg       0.85      0.81      0.82      1768

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      1547
           1       0.40      0.38      0.39       221

    accuracy                           0.85      1768
   macro avg       0.65      0.65      0.65      1768
weighted avg       0.85      0.85      0.85      1768



In [ ]:
len(df2)

1019

In [ ]:
print(classification_report(df1['real'], (df1['pred_all'] > 0.76).astype(int)))
print(classification_report(df2['real'],(df2['pred_all'] > 0.49).astype(int)))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2359
           1       0.37      0.43      0.40       337

    accuracy                           0.84      2696
   macro avg       0.65      0.66      0.65      2696
weighted avg       0.85      0.84      0.84      2696

              precision    recall  f1-score   support

           0       0.92      0.88      0.90      2359
           1       0.36      0.48      0.41       337

    accuracy                           0.83      2696
   macro avg       0.64      0.68      0.65      2696
weighted avg       0.85      0.83      0.84      2696



In [ ]:
print(classification_report(df1['real'], df1['pred']))
# print(classification_report(df2['real'],(df2['pred_all'] > 0.25).astype(int)))
print(classification_report(df2['real'], df2['pred']))
# lstm_treatment_depression 0.4
# lstm_diagnosis_depression 0.5
# lstm_treatment_anxiety 0.5
# lstm_diagnosis_anxiety 0.25

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      2177
           1       0.27      0.32      0.29       311

    accuracy                           0.81      2488
   macro avg       0.59      0.60      0.59      2488
weighted avg       0.82      0.81      0.81      2488

              precision    recall  f1-score   support

           0       0.95      0.86      0.90      2177
           1       0.40      0.67      0.50       311

    accuracy                           0.83      2488
   macro avg       0.67      0.76      0.70      2488
weighted avg       0.88      0.83      0.85      2488



In [ ]:
main_mcnemar(df1['pred'] == df1['real'], df2['pred']== df2['real'])

stat=21.307, p=0.0000039 p<0.001


In [ ]:
main_mcnemar((df1['pred_all'] > 0.40).astype(int) == df1['real'], df2['pred'] == df2['real'])

stat=2.232, p=0.1351663 not significant


In [ ]:
main_mcnemar((df1['pred'] == df1['real']).astype(int), (df2['pred']== df2['real']).astype(int))

stat=1.289, p=0.2561796 not significant


In [ ]:
def compare_models(table, model):
    result = mcnemar(table, exact=True)
    # summarize the finding
    print('\n\n', model)
    # print('statistic=%.5f, p-value=%.5f' % (result.statistic, result.pvalue))
    # # interpret the p-value
    # alpha = 0.05
    # if result.pvalue > alpha:
    #     print('Same proportions of errors (fail to reject H0)')
    # else:
    #     print('Different proportions of errors (reject H0)')
    print('stat=%.3f, p=%.7f' % (result.statistic, result.pvalue),end='')

    significant = False
    for alpha in [0.001, 0.01, 0.05]:
        if result.pvalue < alpha:
            print(' p<' + str(alpha))
            significant = True
            break
    if not significant:
        print(' not significant')


In [ ]:
# def compare_models(table, model):
#     result = mcnemar(table, exact=True)
#     # summarize the finding
#     print('\n\n', model)
#     print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
#     # interpret the p-value
#     alpha = 0.05
#     if result.pvalue > alpha:
#         print('Same proportions of errors (fail to reject H0)')
#     else:
#         print('Different proportions of errors (reject H0)')


In [ ]:
def load_data(task, model):
    # PATH_PRED_BERT = f"/content/drive/MyDrive/doutorado/dados_v6/full/prediction/all_{task}.pkl"
    # df_BERT = pickle.load(open(PATH_PRED_BERT, 'rb'))
    # print(df_lstm)
    # PATH_PRED_RL = f"/content/drive/MyDrive/doutorado/dados_v6/rl/{task}-rl-tfidf.pkl"
    # df_rl = pickle.load(open(PATH_PRED_RL, 'rb'))

    # PATH_PRED_lstm = f"/content/drive/MyDrive/doutorado/dados_v6/lstm/predictions/{task}-LSTM.pkl"
    # df_lstm = pickle.load(open(PATH_PRED_lstm, 'rb'))

    # PATH_PRED_NMT = f"/content/drive/MyDrive/doutorado/dados_v6/full/prediction/medical_terms_{task}.pkl"
    # df_nmt = pickle.load(open(PATH_PRED_NMT, 'rb'))

    # return df_nmt
    # print(classification_report(df_nmt['real'], df_nmt['pred']))

    # PATH_PRED_BERT = f"/content/drive/MyDrive/doutorado/dados_v6/full/prediction/all_{task}.pkl"
    # df_bert = pickle.load(open(PATH_PRED_BERT, 'rb'))

    PATH_PRED_dev80 = f"/content/drive/MyDrive/doutorado/dados_v6/dev80/prediction/A_lstm_balanced_False.pkl"
    df_80 = pickle.load(open(PATH_PRED_dev80, 'rb'))

    PATH_PRED_dev80r = f"/content/drive/MyDrive/doutorado/dados_v6/full/prediction/r80_anxiety.pkl"
    df_80r = pickle.load(open(PATH_PRED_dev80r, 'rb'))

    # PATH_PRED_rafael = f"/content/drive/MyDrive/doutorado/dados_v6/rafael/testing_anxiety.xlsx"
    # df_rafael = pd.read_excel(PATH_PRED_rafael)

    # PATH_PRED_pos = f"/content/drive/MyDrive/doutorado/dados_v6/pre_pos/prediction/D_pos.pkl"
    # df_pos = pickle.load(open(PATH_PRED_pos, 'rb'))

    # PATH_PRED_pre = f"/content/drive/MyDrive/doutorado/dados_v6/pre_pos/prediction/D_pre.pkl"
    # df_pre = pickle.load(open(PATH_PRED_pre, 'rb'))

    print(classification_report(df_80['real'], df_80['pred']))
    print(classification_report(df_80r['real'], df_80r['pred']))
    table = pd.crosstab(df_80['pred'] == df_80['real'], df_80r['pred'] == df_80r['real'])
    compare_models(table, model)
    # return df_lstm, df_bert, table


In [ ]:
PATH_PRED_NMT = f"/content/drive/MyDrive/doutorado/dados_v6/full/prediction/pref_medical_terms_depression.pkl"
df_nmt = pickle.load(open(PATH_PRED_NMT, 'rb'))

In [ ]:
print(classification_report(df_nmt['real'], df_nmt['pred']))

              precision    recall  f1-score   support

           0       0.93      0.79      0.85      2359
           1       0.28      0.58      0.38       337

    accuracy                           0.76      2696
   macro avg       0.60      0.68      0.61      2696
weighted avg       0.85      0.76      0.79      2696



In [ ]:
# load_data('anxiety', 'BERT x LSTM: anxiety')
load_data('anxiety', 'BERT x LSTM: anxiety')
# load_data('anxiety', 'BERT x RL: anxiety')
# load_data('depression', 'BERT x RL: depression')

# load_data('anxiety', 'PRE x POS: anxiety')
# load_data('anxiety', 'RL x BERT: anxiety')

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      3108
           1       0.29      0.18      0.22       444

    accuracy                           0.84      3552
   macro avg       0.59      0.56      0.57      3552
weighted avg       0.81      0.84      0.83      3552

              precision    recall  f1-score   support

           0       0.90      0.84      0.87      3108
           1       0.23      0.33      0.27       444

    accuracy                           0.78      3552
   macro avg       0.57      0.59      0.57      3552
weighted avg       0.81      0.78      0.80      3552



 BERT x LSTM: anxiety
stat=205.000, p=0.0000000 p<0.001


In [ ]:
df_nmt

,pred,real
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
3547,0,1
3548,1,1
3549,1,1
3550,0,1


In [ ]:
table

col_0,False,True
row_0,,
False,184,824
True,480,2064


In [ ]:
df_bert

,pred,real
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
3547,0,1
3548,1,1
3549,1,1
3550,0,1
